In [1]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
# import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform

print(tf.__version__)

2.17.0


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


In [3]:
print(x_train[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

In [ ]:
sns.countplot(y_train)

In [4]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

np.isnan(x_train).any()
np.isnan(x_test).any()

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


False

In [8]:
# SKIP

input_shape = (28, 28, 1)

x_train=x_train.reshape(x_train.shape[0], 28, 28, 1)
old_100 = x_train[100]

x_train = transform.resize(x_train, (x_train.shape[0], 16, 16, 1))
x_train=x_train / 255.0
x_test = transform.resize(x_test, (x_test.shape[0], 16, 16, 1))
x_test=x_test/255.0

In [5]:
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)


In [6]:
plt.figure()
plt.imshow(old_100[:,:,0])
plt.figure()
plt.imshow(x_train[100][:,:,0])
# plt.show()

print(y_train[100])
print(x_train[100].shape)

NameError: name 'old_100' is not defined

<Figure size 640x480 with 0 Axes>

In [7]:
plt.imshow(x_train[1][:,:,0])

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
x_train_flat = x_train.reshape(x_train.shape[0], -1)  # Reshapes to (60000, 784)
x_test_flat = x_test.reshape(x_test.shape[0], -1)  # Reshapes to (10000, 784)

In [ ]:
batch_size = 64
num_classes = 10
epochs = 5

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.legacy.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.995):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
history = model.fit(x_train_flat, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.1,
                    callbacks=[callbacks])

In [ ]:
quantized_model = tfmot.quantization.keras.quantize_model(model)
quantized_model.compile(optimizer=tf.keras.optimizers.legacy.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
quantized_history = quantized_model.fit(x_train_flat, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.1,
                    callbacks=[callbacks])

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='r', label="Validation Loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation Accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
print(quantized_model.predict(x_test_flat[0:1]))
print(x_test_flat[0:1].shape)
test = x_test_flat[0:1].reshape(1, 16, 16)
plt.imshow(test[0])

In [ ]:
np.set_printoptions(threshold=np.inf)

x = False

with open("weights_2.txt", "w") as f:
    for layer in model.layers:
        if not x:
            print(layer.get_weights())
            x = True
        weights, biases = layer.get_weights()
        f.write(f"Weights for layer {layer.name}: {weights}\n")
        f.write(f"Biases for layer {layer.name}: {biases}\n")

In [ ]:
# quantized_model = 

In [ ]:
np.set_printoptions(threshold=np.inf)

with open("weights_quantized.txt", "w") as f:
    for layer in quantized_model.layers:
        try:
            weights, biases = layer.get_weights()
            f.write(f"Weights for layer {layer.name}: {weights}\n")
            f.write(f"Biases for layer {layer.name}: {biases}\n")
        except: 
            pass
        finally:
            print(layer.get_weights())
            
        

In [ ]:
np.set_printoptions(threshold=np.inf)

np.save("x_train", x_train)

In [ ]:
def quantize_weights(weights, num_bits=8):
    min_val = np.min(weights)
    max_val = np.max(weights)
    scale = (max_val - min_val) / (2**num_bits - 1)
    zero_point = -min_val / scale
    quantized_weights = np.round(weights / scale + zero_point).astype(np.int32)
    return quantized_weights, scale, zero_point

In [ ]:
quantized_model = tfmot.quantization.keras.quantize_apply(model)